<a href="https://colab.research.google.com/github/katarinagresova/benchmarks/blob/main/enhancers/Enhancers_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is working with data from paper [Enhancer Identification using Transfer and Adversarial Deep Learning of DNA Sequences](https://www.biorxiv.org/content/biorxiv/early/2018/02/14/264200.full.pdf). 

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip3 install biopython

     |████████████████████████████████| 2.3MB 2.7MB/s 


In [3]:
import urllib
from fastai.text import *
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import os

In [14]:
u = 'http://www.cs.huji.ac.il/~tommy//enhancer_CNN/Enhancers_vs_negative.tgz'
path = Path('/content/drive/My Drive/Benchmarks/enhancers/Enhancers_vs_negative.tgz')
urllib.request.urlretrieve(u, path)

(PosixPath('/content/drive/My Drive/Benchmarks/enhancers/Enhancers_vs_negative.tgz'),
 <http.client.HTTPMessage at 0x7f2b1bb54390>)

In [24]:
!mkdir /content/drive/My\ Drive/Benchmarks/enhancers/unpacked/

In [ ]:
!tar zxvf /content/drive/My\ Drive/Benchmarks/enhancers/Enhancers_vs_negative.tgz -C /content/drive/My\ Drive/Benchmarks/enhancers/unpacked/

In [29]:
def txt_to_fasta(txt_file, fasta_file):
  with open(txt_file, 'r') as input:
    with open(fasta_file, 'w') as output:
      index = 0
      for sequence in input:
        SeqIO.write(
          SeqRecord(
            #sequence[:-1] to remove trailing \n
            Seq(sequence[:-1]), 
            'seq_' + str(index), description=""
          ), 
          output, 
          'fasta'
        )
        index += 1

In [20]:
from pathlib import Path
def make_dir(dir):
  Path(dir).mkdir(parents=True, exist_ok=True)

In [ ]:
in_dir = '/content/drive/My Drive/Benchmarks/enhancers/unpacked/'
out_dir = '/content/drive/My Drive/Benchmarks/enhancers/prepared/'
enhancer_file = 'positive_samples'
negative_file = 'negative_samples'
exclude_dir = 'peaks_fasta_files'
for (root, subdirectories, files) in os.walk(in_dir):
  for subdirectory in subdirectories:
    if subdirectory != exclude_dir:
      current_in_dir = os.path.join(root, subdirectory)
      current_out_dir = os.path.join(out_dir, subdirectory)
      make_dir(current_out_dir)
      txt_to_fasta(current_in_dir + '/' + enhancer_file, current_out_dir + '/' + enhancer_file + ".fa")
      txt_to_fasta(current_in_dir + '/' + negative_file, current_out_dir + '/' + negative_file + ".fa")